#####################################################################################
#
#    Quantum Gradient Descent Tutorial 
#
#
#
#
#
#
#
#
#
#
######################################################################################


In [34]:
#import packages
import numpy as np
import pennylane as qml
from pennylane import expval, var
pi = np.pi

In [35]:
dev = qml.device("default.qubit", wires=2)

#@qml.qnode(dev, interface="autograd")

In [36]:
alpha = 0.4
beta = 0.2

coeffs = [alpha, alpha, beta]

#initialize constants
max_iterations = 500
step_size = 0.05
conv_tol = 1e-06

optimizers = ['QNG', 'Vanilla', 'QNG_diag']
QNG_cost = []
vanilla_cost = []
QNG_diag_cost = []

params = np.array([-0.2,-0.2,0,0])


In [37]:
#create ansatz 
def ansatz(params, wires):
    
    #parameterized layer 0
    qml.RY(params[0], wires=0)
    qml.RY(params[1], wires=1)
    
    #entanglement
    qml.CNOT(wires=[0,1])
    
    #param layer 1
    qml.RY(params[2], wires=0)
    qml.RY(params[3], wires=1)
    


In [38]:
#H2 hamiltonian from Yamamoto (2019)
obs_list = [
    qml.PauliZ(0) @ qml.Identity(1),
    qml.Identity(0) @ qml.PauliZ(1),
    qml.PauliX(0) @ qml.PauliX(1)
]

#create qnodes
qnodes = qml.map(ansatz, obs_list, dev, measure='expval')

#create cost function
cost = qml.dot(coeffs, qnodes)

prev_energy = cost(params)

In [42]:
for k in range(len(optimizers)):
    #Quantum Natural Gradient Descent
    if optimizers[k] == 'QNG':
        print("QUANTUM NATURAL GRADIENT")
        
        #initial params (example from Yamamoto 2019, FIG 5)
        params = np.array([-0.2,-0.2,0,0])
        for n in range(max_iterations):
            #compute the gradient
            gradient = qml.grad(cost, argnum=[0])
            
            #take the gradient of the params
            grad_at_point = [float(i) for i in gradient(params)[0]]
            
            #perform quantum natural gradient.
            params = params - step_size * np.dot(
                
                #np.linalg.pinv(qml.metric_tensor(obs_list)),
                np.linalg.pinv(qnodes[0].metric_tensor([params])), 
                grad_at_point)
            
            
            
            #compute new energy with updates params
            energy = cost(params)
            QNG_cost.append(energy)
            
            #calculate convergence
            conv = np.abs(energy - prev_energy)

            if conv <= conv_tol:
                print('found ground-state at step {:}'.format(n))
                print("Groud-state energy = ", energy)
                break

            prev_energy = energy
            
    #Vanilla Gradient Descent
    if optimizers[k] == 'Vanilla':
        print("\nVANILLA GRADIENT")
        params = np.array([-0.2,-0.2,0,0])
        #params = np.array([(7*pi)/32,pi/2,0,0])
        
        for n in range(max_iterations):
            
            gradient = qml.grad(cost, argnum=[0])
            grad_at_point = [float(i) for i in gradient(params)[0]]
            
            #calculate vanilla gradient descent
            params = params - np.dot(step_size, grad_at_point)
            
            #calculate new energy with updates params
            energy = cost(params)
            vanilla_cost.append(energy)
            
            #calculate convergence
            conv = np.abs(energy - prev_energy)

            if conv <= conv_tol:
                print('found ground-state at step {:}'.format(n))
                print("Groud-state energy = ", energy)
                break

            prev_energy = energy
            
    #QNG Diagonal Approx. Descent
    if optimizers[k] == 'QNG_diag':
        print("\nQUANTUM NATURAL GRADIENT -- DIAGONAL")
        
        #initial params (example from Yamamoto 2019, FIG 5)
        params = np.array([-0.2,-0.2,0,0])
        #params = np.array([(7*pi)/32,pi/2,0,0])
        
        for n in range(max_iterations):
            #compute the gradient
            gradient = qml.grad(cost, argnum=[0])
            
            #take the gradient of the params
            grad_at_point = [float(i) for i in gradient(params)[0]]
            
            #perform quantum natural gradient.
            params = params - step_size * np.dot(
                np.linalg.pinv(qnodes[0].metric_tensor([params], diag_approx=True)), 
                grad_at_point)
            
            #compute new energy with updates params
            energy = cost(params)
            QNG_diag_cost.append(energy)
            
            #calculate convergence
            conv = np.abs(energy - prev_energy)


            if conv <= conv_tol:
                print('found ground-state at step {:}'.format(n))
                print("Groud-state energy = ", energy)
                break

            prev_energy = energy


QUANTUM NATURAL GRADIENT


IndexError: index 1 is out of bounds for axis 0 with size 1